# Daily Indicators per Stock

In [1]:
# Import libraries and dependencies
import pandas as pd
import numpy as np
%matplotlib inline



## Read in Fundamental Indicators

In [63]:
free_cash_flow_equity_path = 'Resources/s_FreeCashFlowEquity.csv'
free_cash_flow_equity_df = pd.read_csv(free_cash_flow_equity_path)

gross_margin_path = 'Resources/s_GrossMargin.csv'
gross_margin_df = pd.read_csv(gross_margin_path)

market_cap_path = 'Resources/s_MarketCap.csv'
market_cap_df = pd.read_csv(market_cap_path)

net_income_margin_path = 'Resources/s_NetIncomeMargin.csv'
net_income_margin_df = pd.read_csv(net_income_margin_path)

pb_path = 'Resources/s_PB.csv'
pb_df = pd.read_csv(pb_path)

pe_path = 'Resources/s_PE.csv'
pe_df = pd.read_csv(pe_path)

px_path = 'Resources/s_PX.csv'
px_df = pd.read_csv(px_path)

roa_path = 'Resources/s_ROA.csv'
roa_df = pd.read_csv(roa_path)

roe_path = 'Resources/s_ROE.csv'
roe_df = pd.read_csv(roe_path)

universe_path = 'Resources/s_universe.csv'
universe_df = pd.read_csv(universe_path)

C:\Users\ekuo651\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [38]:
index_dates_df = pd.read_csv(px_path)

In [13]:
fundamental_indicators = [free_cash_flow_equity_df,
                          gross_margin_df,
                          market_cap_df,
                          net_income_margin_df,
                          pb_df,
                          pe_df,
                          px_df,
                          roa_df,
                          roe_df]
sectors_df = universe_df

In [66]:
index_dates = list(px_df['Date'])
index_dates[:3]

['1/1/1998', '1/2/1998', '1/5/1998']

In [18]:
[print(i.shape) for i in fundamental_indicators]

(5724, 505)
(5724, 505)
(5724, 505)
(5724, 505)
(263, 505)
(5724, 505)
(5724, 505)
(5724, 505)
(263, 505)


[None, None, None, None, None, None, None, None, None]

In [67]:
def drop_date(df):
    return df.drop(columns='Date', inplace=True)

In [68]:
[drop_date(i) for i in fundamental_indicators]

KeyError: "['Date'] not found in axis"

In [17]:
stocks = list(px_df.columns)
stocks[:3]

['ILMN UW Equity', 'EIX UN Equity', 'GPS UN Equity']

In [26]:
def new_df_names(i):
    new_name = 'df_'+str(i)
    return new_name

In [27]:
stock_df_list = [new_df_names(i) for i in stocks]
stock_df_list[:5]

['df_ILMN UW Equity',
 'df_EIX UN Equity',
 'df_GPS UN Equity',
 'df_ES UN Equity',
 'df_NLOK UW Equity']

## Make DataFrames for Individual Stocks

In [45]:
column_names = ['free_cash_flow_equity',
               'gross_margin',
               'market_cap',
               'net_income_margin',
               'pe',
               'px',
               'roa']


def make_df(stock):
    new_df = pd.concat([free_cash_flow_equity_df[stock],
                    gross_margin_df[stock], 
                    market_cap_df[stock],
                    net_income_margin_df[stock],
                    pe_df[stock],
                    px_df[stock],
                    roa_df[stock]],
                   axis='columns')
    new_df.columns = column_names
    new_df.index = index_dates
    return new_df


In [82]:
# dropna(axis=1,how='all') drops columns that are completely null
# dropna() will drop rows with any remaining nulls, essentially the null period before fundamentals were available

make_df(stocks[450]).ffill().dropna(axis=1,how='all').dropna()


,free_cash_flow_equity,market_cap,net_income_margin,pe,px,roa
7/1/1998,49.727,2906.6080,19.1075,19.6910,0.8984,15.3831
7/2/1998,49.727,2221.1815,19.1075,15.0475,0.9115,15.3831
7/3/1998,49.727,2266.0527,19.1075,15.3409,0.9115,15.3831
7/6/1998,-28.386,2007.6943,16.3669,13.7649,0.9531,15.1054
7/7/1998,-28.386,2251.8846,16.3669,15.4391,0.9609,15.1054
7/8/1998,-28.386,2469.9365,16.3669,16.9476,0.9740,15.1054
7/9/1998,100.062,2614.0023,20.5305,18.3556,0.9896,14.4705
7/10/1998,100.062,2555.7098,20.5305,17.9463,0.9948,14.4705
7/13/1998,100.062,2916.5513,20.5305,20.2737,1.0026,14.4705
7/14/1998,113.933,2970.3902,18.7846,21.3987,0.9844,13.8553


In [ ]:
window_year=252
window_month=21
window_quarter=63

def calculate_technical_indicators(i):
    window_year=252
window_month=21
window_quarter=63
    stock_df = make_df(stock[i]).ffill().dropna(axis=1,how='all').dropna()
    stock_df['Price Momentum 12M']=stock_df.px/stock_df.px.shift()
    

## Calculating Technical Indicators

* pct_change (only on the stock price)
*

s_feat1=s_price/s_price.shift(window_year)-1 #Price momentum 12M 
s_feat2=s_price.shift(window_month)/s_price.shift(window_year)-1 #Price momentum 12M-1M 
s_feat3=s_price/s_price.rolling(window_year).max() #Price to 52W High 
s_feat4=s_price/s_price.rolling(window_year).min() #Price to 52W low 
s_feat5=s_price/s_price.shift(window_month)-1 #Return 1M
s_feat6=s_price/s_price.shift(window_quarter)-1 #Return 1Q
s_feat7=s_price/s_price.shift(window_year*5)-1 #Return 60M
s_feat8=s_ret.rolling(window_month).max() #Maximum daily return over 1M
s_feat9=s_ret.rolling(window_month).std() #Realized vol 1M
s_feat10=s_ret.rolling(window_quarter).std() #Realized vol 3M

In [ ]:
s_feat1=s_price/s_price.shift(window_year)-1 #Price momentum 12M 
s_feat2=s_price.shift(window_month)/s_price.shift(window_year)-1 #Price momentum 12M-1M 
s_feat3=s_price/s_price.rolling(window_year).max() #Price to 52W High 
s_feat4=s_price/s_price.rolling(window_year).min() #Price to 52W low 
s_feat5=s_price/s_price.shift(window_month)-1 #Return 1M
s_feat6=s_price/s_price.shift(window_quarter)-1 #Return 1Q
s_feat7=s_price/s_price.shift(window_year*5)-1 #Return 60M
s_feat8=s_ret.rolling(window_month).max() #Maximum daily return over 1M
s_feat9=s_ret.rolling(window_month).std() #Realized vol 1M
s_feat10=s_ret.rolling(window_quarter).std() #Realized vol 3M

In [84]:
for i in stocks:
    variable = make_df(i).ffill().dropna(axis=1,how='all').dropna()
    print(variable) 

            free_cash_flow_equity  gross_margin  market_cap  \
7/28/2000                   218.0       71.4443    47151.72   
7/31/2000                   218.0       71.4443    47151.72   
8/1/2000                    218.0       71.4443    47151.72   
8/2/2000                    218.0       71.4443    47151.72   
8/3/2000                    218.0       71.4443    47151.72   
...                           ...           ...         ...   
12/4/2019                   218.0       71.4443    47151.72   
12/5/2019                   218.0       71.4443    47151.72   
12/6/2019                   218.0       71.4443    47151.72   
12/9/2019                   218.0       71.4443    47151.72   
12/10/2019                  218.0       71.4443    47151.72   

            net_income_margin       pe       px      roa  
7/28/2000             30.2174  53.7752   19.586  12.7894  
7/31/2000             30.2174  53.7752   17.500  12.7894  
8/1/2000              30.2174  53.7752   17.750  12.7894  
8/2/200

KeyboardInterrupt: 